In [66]:
from keras.layers import Input, LSTM, Dense, Conv1D, concatenate
from keras.models import Model
import tensorflow as tf
import pandas as pd


BATCH_SIZE = 100

In [67]:
#data
file_path = "D:\GitHub\P9-Exoskeleton\Datasets\DataArk19.csv"

df = pd.read_csv(file_path)

TRAIN_SPLIT = round(len(df)/2)

print(TRAIN_SPLIT)
df.head()

In [68]:
columns_data = ['s_1' ,'s_2', 's_3', 's_4', 's_5', 's_6', 's_7', 's_8','angle']

batches = tf.data.experimental.make_csv_dataset(file_path, batch_size=BATCH_SIZE,label_name="angle",select_columns=columns_data)

In [69]:
#input

inputs = Input(shape=(50,8), name='main_input')

conv = Conv1D(filters=20,kernel_size=5,strides=2,padding='valid',activation='relu')(inputs)

lstm = LSTM(32)(conv)

auxiliary_output = Dense(1, activation='sigmoid', name='aux_output')(lstm)

print("success")

In [70]:
#intermediate layers
aux_input = Input(shape=(5,), name='aux_input')

concat = concatenate([lstm,aux_input])

x = Dense(64, activation='relu')(concat)
x = Dense(64, activation='relu')(x)
x = Dense(64, activation='relu')(x)

main_output = Dense(1, activation='sigmoid',name='main_output')(x)

print('success')

In [71]:
#model

model = Model(inputs=[inputs,aux_input], outputs=[main_output,auxiliary_output])

model.summary()

model.compile(optimizer=tf.keras.optimizers.RMSprop(clipvalue=1.0), loss={'main_output':'binary_crossentropy',
                                                                          'aux_output':'binary_crossentropy'},
              loss_weights={'main_output':1, 'aux_output':0.2})

model.fit({'main_input': batches, 'aux_input': batches},
          {'main_output': batches, 'aux_output': batches},
          epochs=50, batch_size=BATCH_SIZE)